In [13]:
import os
import boto3
import pandas as pd
from pdf2image import convert_from_path

In [14]:
client = boto3.client('textract')

In [15]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

In [16]:
def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']

In [18]:
for city in os.listdir("Cities/"):
    for document in os.listdir("Cities/"+city+"/"):
        if (document.split(".")[-1] == "pdf"):
            images = convert_from_path("Cities/"+city+"/"+document)
            for i in range(len(images)):
                images[i].save("Cities/"+city+"/"+document.split(".")[0]+'_page'+ str(i) +'.jpg', 'JPEG')

In [22]:
for city in os.listdir("Cities/"):
        for document in os.listdir("Cities/"+city+"/"):
            if (document.split(".")[-1] == "jpg"):
                with open("Cities/"+city+"/"+document, "rb") as file:
                    f = file.read()
                    b = bytearray(f)
                response = client.analyze_document(Document={'Bytes': b}, FeatureTypes=["TABLES"])
                
                blocks = response['Blocks']
                tables = map_blocks(blocks, 'TABLE')
                cells = map_blocks(blocks, 'CELL')
                words = map_blocks(blocks, 'WORD')
                selections = map_blocks(blocks, 'SELECTION_ELEMENT')

                count = 0

                for table in tables.values():

                    count += 1

                    # Determine all the cells that belong to this table
                    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

                    # Determine the table's number of rows and columns
                    n_rows = max(cell['RowIndex'] for cell in table_cells)
                    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
                    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

                    # Fill in each cell
                    for cell in table_cells:
                        cell_contents = [
                            words[child_id]['Text']
                            if child_id in words
                            else selections[child_id]['SelectionStatus']
                            for child_id in get_children_ids(cell)
                        ]
                        i = cell['RowIndex'] - 1
                        j = cell['ColumnIndex'] - 1
                        content[i][j] = ' '.join(cell_contents)

                    # We assume that the first row corresponds to the column names
                    dataframe = pd.DataFrame(content[1:], columns=content[0])
                    dataframe.to_csv("Cities/"+city+"/"+document.split(".")[0]+"_table"+str(count)+".csv", encoding = 'utf-8')